In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [209]:
import pymongo

client = pymongo.MongoClient() 
db = client.sae

In [210]:
import pandas as pd

## 1. Lister les clients n’ayant jamais effecuté une commande 

In [211]:
c = db.customers.aggregate([
    {
        "$match": {
            "$or": [
                {"Order": {"$exists": False}},  # Cas où 'Order' n'existe pas
                {"Order": {"$size": 0}}        # Cas où 'Order' est une liste vide
            ]
        }
    },
    {
        "$project": {  # Sélectionne uniquement les champs pertinents
            "_id": 0,
            "customerNumber": 1,
            "customerName": 1,
            "phone": 1
        }
    }
])

pd.DataFrame(list(c))



,customerNumber,customerName,phone
0,103,Atelier graphique,40.32.2555
1,112,Signal Gift Stores,7025551838
2,114,"Australian Collectors, Co.",03 9520 4555
3,119,La Rochelle Gifts,40.67.8555
4,121,Baane Mini Imports,07-98 9555
...,...,...,...
117,486,Motor Mint Distributors Inc.,2155559857
118,487,Signal Collectibles Ltd.,4155554312
119,489,"Double Decker Gift Stores, Ltd",(171) 555-7555
120,495,Diecast Collectables,6175552555


## 2. Pour chaque employé, le nombre de clients, le nombre de commandes et le montant total de celles-ci


In [212]:
result = db.employees.aggregate([
    # Étape 1 : Jointure avec la collection "customers" pour récupérer les clients d'un employé
    {
        "$lookup": {
            "from": "customers",
            "localField": "employeeNumber",
            "foreignField": "salesRepEmployeeNumber",
            "as": "customers"
        }
    },
    { "$unwind": { "path": "$customers", "preserveNullAndEmptyArrays": True } },

    # Étape 2 : Déplier les commandes des clients
    {
        "$unwind": { "path": "$customers.Orders", "preserveNullAndEmptyArrays": True }
    },

    # Étape 3 : Déplier les détails des commandes
    {
        "$unwind": { "path": "$customers.Orders.Orderdetails", "preserveNullAndEmptyArrays": True }
    },

    # Étape 4 : Calculer le montant total de chaque ligne de commande
    {
        "$addFields": {
            "orderAmount": {
                "$multiply": [
                    { "$ifNull": ["$customers.Orders.Orderdetails.priceEach", 0] },
                    { "$ifNull": ["$customers.Orders.Orderdetails.quantityOrdered", 0] }
                ]
            }
        }
    },

    # Étape 5 : Groupement pour chaque employé
    {
        "$group": {
            "_id": "$employeeNumber",
            "firstName": { "$first": "$firstName" },
            "lastName": { "$first": "$lastName" },
            "numberOfCustomers": { "$addToSet": "$customers.customerNumber" },
            "numberOfOrders": { "$sum": { "$cond": [{ "$ifNull": ["$customers.Orders.orderNumber", False] }, 1, 0] } },
            "totalOrderAmount": { "$sum": "$orderAmount" }
        }
    },

    # Étape 6 : Projeter les résultats finaux
    {
        "$project": {
            "_id": 0,
            "employeeNumber": "$_id",
            "firstName": 1,
            "lastName": 1,
            "numberOfCustomers": { "$size": "$numberOfCustomers" },
            "numberOfOrders": 1,
            "totalOrderAmount": 1
        }
    },

    # Étape 7 : Trier les résultats par numéro d'employé
    { "$sort": { "employeeNumber": 1 } }
])

# Convertir les résultats en DataFrame ou afficher
results = list(result)
df = pd.DataFrame(results)
print(df)


   firstName   lastName  numberOfOrders  totalOrderAmount  employeeNumber  \
0      Diane     Murphy               0              0.00            1002   
1       Mary  Patterson               0              0.00            1056   
2       Jeff   Firrelli               0              0.00            1076   
3    William  Patterson               0              0.00            1088   
4     Gerard     Bondur               0              0.00            1102   
5    Anthony        Bow               0              0.00            1143   
6     Leslie   Jennings             331        1210228.57            1165   
7     Leslie   Thompson             114         378064.72            1166   
8      Julie   Firrelli             124         422257.44            1188   
9      Steve  Patterson             152         565516.73            1216   
10  Foon Yue      Tseng             142         550395.19            1286   
11    George     Vanauf             211         725598.76            1323   

# 3. Idem pour chaque bureau (nombre de clients, nombre de commandes et montant total), avec en plus le nombre de clients d’un pays différent, s’il y en a

In [213]:
office_stats = db.employees.aggregate([
    # Jointure avec la collection customers pour récupérer les clients associés à chaque employé
    {
        "$lookup": {
            "from": "customers",
            "localField": "employeeNumber",
            "foreignField": "salesRepEmployeeNumber",
            "as": "customers"
        }
    },
    { "$unwind": { "path": "$customers", "preserveNullAndEmptyArrays": True } },

    # Ajout des informations sur les commandes à partir des clients
    {
        "$addFields": {
            "customers.orders": {
                "$ifNull": ["$customers.Orders", []]
            }
        }
    },

    # Calcul du montant total des commandes et du nombre de commandes pour chaque client
    {
        "$addFields": {
            "orderAmount": {
                "$sum": {
                    "$map": {
                        "input": "$customers.orders",
                        "as": "order",
                        "in": {
                            "$sum": {
                                "$map": {
                                    "input": "$$order.Orderdetails",
                                    "as": "detail",
                                    "in": {
                                        "$multiply": ["$$detail.priceEach", "$$detail.quantityOrdered"]
                                    }
                                }
                            }
                        }
                    }
                }
            },
            "orderCount": { "$size": "$customers.orders" }
        }
    },

    # Groupement par bureau (officeCode)
    {
        "$group": {
            "_id": "$officeCode",
            "city": { "$first": "$Offices.city" },
            "officeCountry": { "$first": "$Offices.country" },
            "numberOfCustomers": { "$addToSet": "$customers.customerNumber" },
            "totalOrders": { "$sum": "$orderCount" },
            "totalOrderAmount": { "$sum": "$orderAmount" },
            "customersFromDifferentCountry": {
                "$addToSet": {
                    "$cond": [
                        { "$ne": ["$customers.country", "$Offices.country"] },
                        "$customers.customerNumber",
                        None
                    ]
                }
            }
        }
    },

    # Calcul du nombre réel de clients et de clients d'un pays différent
    {
        "$project": {
            "_id": 0,
            "officeCode": "$_id",
            "city": 1,
            "officeCountry": 1,
            "numberOfCustomers": { "$size": "$numberOfCustomers" },
            "totalOrders": 1,
            "totalOrderAmount": 1,
            "customersFromDifferentCountry": {
                "$size": {
                    "$setDifference": ["$customersFromDifferentCountry", [None]]
                }
            }
        }
    },

    # Tri par officeCode
    { "$sort": { "officeCode": 1 } }
])

# Convertir en DataFrame
results = list(office_stats)
df = pd.DataFrame(results)
print(df)



              city officeCountry  totalOrders  totalOrderAmount officeCode  \
0  [San Francisco]         [USA]           48        1588293.29        1.0   
1         [Boston]         [USA]           32         987774.17        2.0   
2            [NYC]         [USA]           39        1275993.95        3.0   
3          [Paris]      [France]          106        3404055.56        4.0   
4          [Tokyo]       [Japan]           16         503957.58        5.0   
5         [Sydney]   [Australia]           38        1281705.83        6.0   
6         [London]          [UK]           47        1604168.80        7.0   

   numberOfCustomers  customersFromDifferentCountry  
0                 12                             12  
1                 12                             12  
2                 15                             15  
3                 29                             29  
4                  5                              5  
5                 10                             10

# 4. Pour chaque produit, donner le nombre de commandes, la quantité totale commandée, et le nombre de clients différents

In [221]:
# Pipeline pour agrégation de produits et commandes
pipeline = [
    {
        "$unwind": "$Orders"  # Décomposer les orders dans customers
    },
    {
        "$unwind": "$Orders.Orderdetails"  # Décomposer les orderdetails dans chaque order
    },
    {
        "$lookup": {
            "from": "products",  # La collection des produits
            "localField": "Orders.Orderdetails.productCode",  # Relier les produits par productCode
            "foreignField": "productCode",
            "as": "product_info"
        }
    },
    {
        "$unwind": "$product_info"  # Décomposer la liste des produits
    },
    {
        "$group": {
            "_id": "$product_info.productCode",  # Groupement par productCode
            "productName": { "$first": "$product_info.productName" },
            "numberOfOrders": { "$sum": 1 },  # Compter le nombre d'orders pour chaque produit
            "totalQuantityOrdered": { "$sum": "$Orders.Orderdetails.quantityOrdered" },  # Somme des quantités
            "numberOfDistinctCustomers": { "$addToSet": "$customerNumber" }  # Liste des clients distincts
        }
    },
    {
        "$project": {
            "_id": 0,
            "productCode": "$_id",
            "productName": 1,
            "numberOfOrders": 1,
            "totalQuantityOrdered": 1,
            "numberOfDistinctCustomers": { "$size": "$numberOfDistinctCustomers" }  # Compter les clients distincts
        }
    },
    {
        "$sort": { "productCode": 1 }  # Trier par productCode
    }
]

# Exécuter l'agrégation sur la base de données MongoDB
results = db.customers.aggregate(pipeline)

# Convertir les résultats en DataFrame pour visualisation
df_results = pd.DataFrame(list(results))
print(df_results)


                               productName  numberOfOrders  \
0    1969 Harley Davidson Ultimate Chopper              28   
1                 1952 Alpine Renault 1300              28   
2                    1996 Moto Guzzi 1100i              28   
3     2003 Harley-Davidson Eagle Drag Bike              28   
4                      1972 Alfa Romeo GTA              28   
..                                     ...             ...   
104                            The Titanic              27   
105                         The Queen Mary              27   
106              American Airlines: MD-11S              28   
107                       Boeing X-32A JSF              28   
108                             Pont Yacht              27   

     totalQuantityOrdered productCode  numberOfDistinctCustomers  
0                    1026    S10_1678                         26  
1                     961    S10_1949                         27  
2                     999    S10_2016                 

## 5. Donner le nombre de commande pour chaque pays, ainsi que le montant total des commandes et le montant total payé : on veut conserver les clients n’ayant jamais commandé dans le résultat final


In [223]:
office_stats = db.employees.aggregate([
    # Jointure avec la collection customers pour récupérer les clients associés à chaque employé
    {
        "$lookup": {
            "from": "customers",
            "localField": "employeeNumber",
            "foreignField": "salesRepEmployeeNumber",
            "as": "customers"
        }
    },
    { "$unwind": { "path": "$customers", "preserveNullAndEmptyArrays": True } },

    # Ajout des informations sur les commandes à partir des clients
    {
        "$addFields": {
            "customers.orders": {
                "$ifNull": ["$customers.Orders", []]  # On garde les clients sans commandes
            }
        }
    },

    # Ajout d'une jointure avec la collection de paiements pour récupérer les informations de paiement
    {
        "$lookup": {
            "from": "payments",
            "localField": "customers.customerNumber",
            "foreignField": "customerNumber",
            "as": "payments"
        }
    },

    # Calcul du montant total des commandes et du nombre de commandes pour chaque client
    {
        "$addFields": {
            "orderAmount": {
                "$sum": {
                    "$map": {
                        "input": "$customers.orders",
                        "as": "order",
                        "in": {
                            "$sum": {
                                "$map": {
                                    "input": "$$order.Orderdetails",
                                    "as": "detail",
                                    "in": {
                                        "$multiply": ["$$detail.priceEach", "$$detail.quantityOrdered"]
                                    }
                                }
                            }
                        }
                    }
                }
            },
            "orderCount": { "$size": "$customers.orders" },
            "totalAmountPaid": {
                "$sum": "$payments.amount"  # Somme des paiements effectués pour chaque client
            }
        }
    },

    # Groupement par pays
    {
        "$group": {
            "_id": "$customers.country",  # Regroupement par pays du client
            "numberOfCustomers": { "$addToSet": "$customers.customerNumber" },
            "totalOrders": { "$sum": "$orderCount" },
            "totalOrderAmount": { "$sum": "$orderAmount" },
            "totalAmountPaid": { "$sum": "$totalAmountPaid" }
        }
    },

    # Calcul du nombre réel de clients et des informations finales
    {
        "$project": {
            "_id": 0,
            "country": "$_id",
            "numberOfCustomers": { "$size": "$numberOfCustomers" },
            "totalOrders": 1,
            "totalOrderAmount": 1,
            "totalAmountPaid": 1
        }
    },

    # Tri par pays (country)
    { "$sort": { "country": 1 } }
])

# Convertir les résultats en DataFrame
results = list(office_stats)
df = pd.DataFrame(results)
print(df)


    totalOrders  totalOrderAmount  totalAmountPaid      country  \
0             0              0.00             0.00         None   
1            19         630623.10        566642.90    Australia   
2             7         202062.53        134939.48      Austria   
3             7         108412.62        100000.67      Belgium   
4             7         224078.56        190385.59       Canada   
5             7         245637.15        219624.28      Denmark   
6             9         329581.91        329581.91      Finland   
7            37        1110916.56       1017384.70       France   
8             7         220472.09        220472.09      Germany   
9             2          48784.36         48784.36    Hong Kong   
10            2          57756.43         57756.43      Ireland   
11           10         403626.22        362116.28        Italy   
12            6         188167.81        176527.25        Japan   
13           15         535584.00        419845.44  New Zealan

# 6. On veut la table de contigence du nombre de commande entre la ligne de produits et le pays du client 

In [224]:
office_stats = db.employees.aggregate([
    # Jointure avec la collection customers pour récupérer les clients associés à chaque employé
    {
        "$lookup": {
            "from": "customers",
            "localField": "employeeNumber",
            "foreignField": "salesRepEmployeeNumber",
            "as": "customers"
        }
    },
    { "$unwind": { "path": "$customers", "preserveNullAndEmptyArrays": True } },

    # Jointure avec la collection orders pour récupérer les commandes des clients
    {
        "$lookup": {
            "from": "orders",
            "localField": "customers.customerNumber",
            "foreignField": "customerNumber",
            "as": "orders"
        }
    },

    # Jointure avec la collection orderdetails pour récupérer les détails de chaque commande
    {
        "$lookup": {
            "from": "orderdetails",
            "localField": "orders.orderNumber",
            "foreignField": "orderNumber",
            "as": "orderdetails"
        }
    },

    # Jointure avec la collection products pour récupérer les lignes de produits
    {
        "$lookup": {
            "from": "products",
            "localField": "orderdetails.productCode",
            "foreignField": "productCode",
            "as": "products"
        }
    },

    # Regroupement par pays et ligne de produit (productLine)
    {
        "$unwind": { "path": "$products", "preserveNullAndEmptyArrays": True } },  # Pour pouvoir récupérer productLine même si certains détails sont vides
    
    {
        "$group": {
            "_id": {
                "country": "$customers.country",  # Regroupement par pays
                "productLine": "$products.productLine"  # Regroupement par ligne de produit
            },
            "numberOfOrders": { "$sum": 1 }  # Nombre de commandes pour chaque combinaison pays/ligne de produit
        }
    },

    # Projet des résultats dans un format plus lisible
    {
        "$project": {
            "_id": 0,
            "country": "$_id.country",  # Pays
            "productLine": "$_id.productLine",  # Ligne de produit
            "numberOfOrders": 1  # Nombre de commandes
        }
    },

    # Tri par pays et ligne de produit
    { "$sort": { "country": 1, "productLine": 1 } }
])

# Convertir les résultats en DataFrame
results = list(office_stats)
df = pd.DataFrame(results)
print(df)


    numberOfOrders      country
0                8          NaN
1                5    Australia
2                2      Austria
3                2      Belgium
4                3       Canada
5                2      Denmark
6                3      Finland
7               12       France
8                3      Germany
9                1    Hong Kong
10               1      Ireland
11               4        Italy
12               2        Japan
13               4  New Zealand
14               3       Norway
15               1  Philippines
16               2    Singapore
17               5        Spain
18               2       Sweden
19               2  Switzerland
20               5           UK
21              36          USA


# 7. On veut la même table croisant la ligne de produits et le pays du client, mais avec le montant total payé dans chaque cellule

In [225]:
office_stats = db.employees.aggregate([
    # Jointure avec la collection customers pour récupérer les clients associés à chaque employé
    {
        "$lookup": {
            "from": "customers",
            "localField": "employeeNumber",
            "foreignField": "salesRepEmployeeNumber",
            "as": "customers"
        }
    },
    { "$unwind": { "path": "$customers", "preserveNullAndEmptyArrays": True } },

    # Jointure avec la collection orders pour récupérer les commandes des clients
    {
        "$lookup": {
            "from": "orders",
            "localField": "customers.customerNumber",
            "foreignField": "customerNumber",
            "as": "orders"
        }
    },

    # Jointure avec la collection orderdetails pour récupérer les détails de chaque commande
    {
        "$lookup": {
            "from": "orderdetails",
            "localField": "orders.orderNumber",
            "foreignField": "orderNumber",
            "as": "orderdetails"
        }
    },

    # Jointure avec la collection products pour récupérer les lignes de produits
    {
        "$lookup": {
            "from": "products",
            "localField": "orderdetails.productCode",
            "foreignField": "productCode",
            "as": "products"
        }
    },

    # Jointure avec la collection payments pour récupérer les paiements des clients
    {
        "$lookup": {
            "from": "payments",
            "localField": "customers.customerNumber",
            "foreignField": "customerNumber",
            "as": "payments"
        }
    },

    # Regroupement par pays et ligne de produit (productLine)
    {
        "$unwind": { "path": "$products", "preserveNullAndEmptyArrays": True } },  # Pour pouvoir récupérer productLine même si certains détails sont vides
    
    {
        "$group": {
            "_id": {
                "country": "$customers.country",  # Regroupement par pays
                "productLine": "$products.productLine"  # Regroupement par ligne de produit
            },
            # Calcul du montant total payé pour chaque combinaison pays/ligne de produit
            "totalPaidAmount": { 
                "$sum": {
                    "$sum": {
                        "$map": {
                            "input": "$payments",
                            "as": "payment",
                            "in": "$$payment.amount"
                        }
                    }
                }
            },
            "numberOfOrders": { "$sum": 1 }  # Nombre de commandes pour chaque combinaison pays/ligne de produit
        }
    },

    # Projet des résultats dans un format plus lisible
    {
        "$project": {
            "_id": 0,
            "country": "$_id.country",  # Pays
            "productLine": "$_id.productLine",  # Ligne de produit
            "totalPaidAmount": 1,  # Montant total payé
            "numberOfOrders": 1  # Nombre de commandes
        }
    },

    # Tri par pays et ligne de produit
    { "$sort": { "country": 1, "productLine": 1 } }
])

# Convertir les résultats en DataFrame
results = list(office_stats)
df = pd.DataFrame(results)
print(df)


    totalPaidAmount  numberOfOrders      country
0              0.00               8          NaN
1         566642.90               5    Australia
2         134939.48               2      Austria
3         100000.67               2      Belgium
4         190385.59               3       Canada
5         219624.28               2      Denmark
6         329581.91               3      Finland
7        1017384.70              12       France
8         220472.09               3      Germany
9          48784.36               1    Hong Kong
10         57756.43               1      Ireland
11        362116.28               4        Italy
12        176527.25               2        Japan
13        419845.44               4  New Zealand
14        307463.70               3       Norway
15         94015.73               1  Philippines
16        285900.83               2    Singapore
17       1087510.36               5        Spain
18        210014.21               2       Sweden
19        117713.56 

# 8. Donner les 10 produits pour lesquels la marge moyenne est la plus importante (cf buyPrice et priceEach) 


In [229]:
# Agrégation pour calculer la marge moyenne des produits
top_products = db.employees.aggregate([
    # Jointure avec la collection customers pour récupérer les clients associés à chaque employé
    {
        "$lookup": {
            "from": "customers",
            "localField": "employeeNumber",
            "foreignField": "salesRepEmployeeNumber",
            "as": "customers"
        }
    },
    { "$unwind": { "path": "$customers", "preserveNullAndEmptyArrays": True } },

    # Accéder aux commandes des clients
    {
        "$addFields": {
            "customers.orders": {
                "$ifNull": ["$customers.Orders", []]
            }
        }
    },

    # Décomposer les commandes et leurs détails pour accéder à chaque produit commandé
    { "$unwind": { "path": "$customers.orders", "preserveNullAndEmptyArrays": True } },
    { "$unwind": { "path": "$customers.orders.Orderdetails", "preserveNullAndEmptyArrays": True } },

    # Jointure avec la collection products pour obtenir les informations des produits
    {
        "$lookup": {
            "from": "products",
            "localField": "customers.orders.Orderdetails.productCode",
            "foreignField": "productCode",
            "as": "product_info"
        }
    },

    # Décomposer le tableau 'product_info' pour récupérer les informations sur chaque produit
    { "$unwind": "$product_info" },

    # Calcul de la marge pour chaque produit
    {
        "$addFields": {
            "marge": {
                "$subtract": ["$customers.orders.Orderdetails.priceEach", "$product_info.buyPrice"]
            }
        }
    },

    # Calcul de la marge moyenne par produit
    {
        "$group": {
            "_id": "$product_info.productCode",
            "productName": { "$first": "$product_info.productName" },
            "averageMargin": { "$avg": "$marge" }
        }
    },

    # Trier les produits par marge moyenne décroissante
    { "$sort": { "averageMargin": -1 } },

    # Limiter les résultats aux 10 produits ayant la marge moyenne la plus élevée
    { "$limit": 10 },

    # Projection des résultats dans un format lisible
    {
        "$project": {
            "_id": 0,
            "productCode": "$_id",
            "productName": 1,
            "averageMargin": 1
        }
    }
])

# Convertir les résultats en DataFrame
results = list(top_products)
df = pd.DataFrame(results)
print(df)


                            productName  averageMargin productCode
0              1952 Alpine Renault 1300      99.006429    S10_1949
1  2003 Harley-Davidson Eagle Drag Bike      95.235000    S10_4698
2           1992 Ferrari 360 Spider red      83.334906    S18_3232
3                      2002 Suzuki XREO      83.201429    S12_2823
4                1928 Mercedes-Benz SSK      82.696786    S18_2795
5                     2001 Ferrari Enzo      81.043704    S12_1108
6                      1969 Ford Falcon      77.335926    S12_3891
7        1948 Porsche Type 356 Roadster      72.636800    S18_3685
8          1999 Indy 500 Monte Carlo SS      71.794400    S18_2870
9              1917 Grand Touring Sedan      70.432800    S18_1749


# 9. Lister les produits (avec le nom et le code du client) qui ont été vendus à perte :
## - Si un produit a été dans cette situation plusieurs fois, il doit apparaître plusieurs fois,
## - Une vente à perte arrive quand le prix de vente est inférieur au prix d’achat ;

In [231]:
# Agrégation pour lister les produits vendus à perte avec les détails complets
loss_sales = db.employees.aggregate([
    # Jointure avec la collection customers pour récupérer les clients associés à chaque employé
    {
        "$lookup": {
            "from": "customers",
            "localField": "employeeNumber",
            "foreignField": "salesRepEmployeeNumber",
            "as": "customers"
        }
    },
    { "$unwind": { "path": "$customers", "preserveNullAndEmptyArrays": True } },

    # Accéder aux commandes des clients
    {
        "$addFields": {
            "customers.orders": {
                "$ifNull": ["$customers.Orders", []]
            }
        }
    },

    # Décomposer les commandes et leurs détails pour accéder à chaque produit commandé
    { "$unwind": { "path": "$customers.orders", "preserveNullAndEmptyArrays": True } },
    { "$unwind": { "path": "$customers.orders.Orderdetails", "preserveNullAndEmptyArrays": True } },

    # Jointure avec la collection products pour obtenir les informations des produits
    {
        "$lookup": {
            "from": "products",
            "localField": "customers.orders.Orderdetails.productCode",
            "foreignField": "productCode",
            "as": "product_info"
        }
    },

    # Décomposer le tableau 'product_info' pour récupérer les informations sur chaque produit
    { "$unwind": "$product_info" },

    # Filtrage des ventes à perte (prix de vente < prix d'achat)
    {
        "$match": {
            "$expr": {
                "$lt": ["$customers.orders.Orderdetails.priceEach", "$product_info.buyPrice"]
            }
        }
    },

    # Projection des résultats (nom du produit, code produit, nom du client, code client, prix de vente, prix d'achat)
    {
        "$project": {
            "_id": 0,
            "productCode": "$product_info.productCode",
            "productName": "$product_info.productName",
            "customerName": "$customers.customerName",
            "customerNumber": "$customers.customerNumber",
            "priceEach": "$customers.orders.Orderdetails.priceEach",
            "buyPrice": "$product_info.buyPrice"
        }
    },

    # Tri par code produit et code client
    { "$sort": { "productCode": 1, "customerNumber": 1 } }
])

# Convertir les résultats en DataFrame
results = list(loss_sales)
df = pd.DataFrame(results)
print(df)


   productCode                            productName  \
0     S10_1678  1969 Harley Davidson Ultimate Chopper   
1     S10_2016                  1996 Moto Guzzi 1100i   
2     S10_2016                  1996 Moto Guzzi 1100i   
3     S10_2016                  1996 Moto Guzzi 1100i   
4     S10_4698   2003 Harley-Davidson Eagle Drag Bike   
..         ...                                    ...   
74    S24_4048      1992 Porsche Cayenne Turbo Silver   
75    S24_4258                  1936 Chrysler Airflow   
76    S32_1374                       1997 BMW F650 ST   
77    S32_4485              1974 Ducati 350 Mk3 Desmo   
78    S50_1392       Diamond T620 Semi-Skirted Tanker   

                    customerName  customerNumber  priceEach  buyPrice  
0              La Rochelle Gifts             119      34.91     48.81  
1      Souveniers And Things Co.             282      51.15     68.99  
2        Tokyo Collectables, Ltd             398      68.92     68.99  
3               FunGiftIdea

# 10. (bonus) Lister les clients pour lesquels le montant total payé est supérieur aux montants totals des achats 

In [232]:
# Agrégation pour lister les clients dont le montant payé est supérieur au montant des achats
paid_more_than_orders = db.customers.aggregate([
    # Jointure avec la collection de paiements pour récupérer les paiements par client
    {
        "$lookup": {
            "from": "payments",
            "localField": "customerNumber",
            "foreignField": "customerNumber",
            "as": "payments"
        }
    },
    
    # Calcul du montant total payé pour chaque client
    {
        "$addFields": {
            "totalPaid": {
                "$sum": "$payments.amount"
            }
        }
    },

    # Ajout des informations sur les commandes à partir des clients
    {
        "$addFields": {
            "totalOrderAmount": {
                "$sum": {
                    "$map": {
                        "input": "$Orders",
                        "as": "order",
                        "in": {
                            "$sum": {
                                "$map": {
                                    "input": "$$order.Orderdetails",
                                    "as": "detail",
                                    "in": {
                                        "$multiply": ["$$detail.priceEach", "$$detail.quantityOrdered"]
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    },

    # Filtrage des clients dont le montant total payé est supérieur au montant des achats
    {
        "$match": {
            "$expr": {
                "$gt": ["$totalPaid", "$totalOrderAmount"]
            }
        }
    },

    # Projection des résultats : inclure le nom du client, le montant payé et le montant des achats
    {
        "$project": {
            "_id": 0,
            "customerNumber": 1,
            "customerName": 1,
            "totalPaid": 1,
            "totalOrderAmount": 1
        }
    },

    # Tri par montant payé décroissant
    { "$sort": { "totalPaid": -1 } }
])

# Convertir les résultats en DataFrame
results = list(paid_more_than_orders)
df = pd.DataFrame(results)
print(df)


   customerNumber             customerName  totalPaid  totalOrderAmount
0             455         Super Scale Inc.   79472.07          79472.07
1             129          Mini Wheels Co.   74476.18          74476.18
2             452          Mini Auto Werke   52263.90          52263.90
3             198  Auto-Moto Classics Inc.   26479.26          26479.26
